In [6]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import unicodedata
import logging
import csv
import time
import re
import json
import pandas as pd
import mysql.connector

db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Jzy969811",
  database="movie_sys"
)

cursor = db.cursor()
movie_usr_rate_path = "info/usr_rate.csv"
movie_final_info_path = "info/movie_final_info.csv"
movie_rate_info_path = "raw_data/ratings_small.csv"

In [2]:
df_mv = pd.read_csv(movie_final_info_path)[['tmdbId']]

df_rate = pd.read_csv(movie_rate_info_path).rename(columns={"movieId":'tmdbId'})
df_usr_rate = pd.merge(df_mv,df_rate, on='tmdbId')
df_usr = df_usr_rate[["userId"]].drop_duplicates(subset='userId')

In [46]:
import random
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Jzy969811",
  database="movie_sys"
)

cursor = db.cursor()

add_data = ("INSERT INTO UserInfo "
            "(user_id, uname, email, phone, tgroup)"
            "VALUES (%s, %s, %s, %s, %s)")
add_account = ("INSERT INTO UserLogin "
            "(user_id, username, pwd)"
            "VALUES (%s, %s, %s)")
phone = 13948395738
pwd = "888888"
for i, r in df_usr.iterrows():
    user_id = r["userId"]
    uname = "user"+str(user_id)
    email = "usr"+str(user_id).zfill(8)+"@mv.com"
    phone = str(phone+user_id)
    data = (str(user_id), uname, email, phone, random.choice(["A","B"]))
    account = (str(user_id), email, pwd)
    phone = int(phone)
    cursor.execute(add_data, data)
    cursor.execute(add_account, account)
db.commit()
cursor.close()
db.close()

In [3]:
from datetime import datetime
for i,row in enumerate(df_usr_rate["timestamp"]):
    dt_object = datetime.fromtimestamp(row)
    formatted_date = dt_object.strftime("%Y-%m-%d %H:%M:%S")
    df_usr_rate.loc[i,"timestamp"] = formatted_date
print(df_usr_rate)

       tmdbId  userId  rating            timestamp
0         949      23     3.5  2006-05-27 17:11:32
1         949     102     4.0  2000-04-25 01:55:42
2         949     232     2.0  2000-04-07 15:31:37
3         949     242     5.0  2000-04-26 02:53:45
4         949     263     3.0  2005-06-04 08:56:15
...       ...     ...     ...                  ...
14529    2436     243     3.0  2004-09-03 22:31:49
14530    2436     312     3.0  2000-06-02 16:20:26
14531    2436     363     4.0  1999-11-12 02:18:23
14532    2436     427     2.0  2000-01-24 04:05:36
14533    2436     529     4.0  2000-06-03 01:15:38

[14534 rows x 4 columns]


In [ ]:
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Jzy969811",
  database="movie_sys"
)
table_name = 'MovieInfo'
add_data = ("INSERT INTO MovieRate "
            "(user_id, movie_id, rating, rate_time)"
            "VALUES (%s, %s, %s, %s)")

cursor = db.cursor()
for i, r in df_usr_rate.iterrows():
    user_id = r["userId"]
    tmdb_id = r["tmdbId"]
    rating = r["rating"]
    rate_time = r["timestamp"]
    query = f"SELECT movie_id FROM {table_name} WHERE tmdb_id = %s"
    cursor.execute(query, (tmdb_id,))
    result = cursor.fetchall()
    movie_id = str(result[0][0])
    data = (user_id,movie_id,rating,rate_time)
    cursor.execute(add_data,data)
db.commit()
cursor.close()
db.close()

In [5]:
df_usr_rate = df_usr_rate.sort_values('timestamp', ascending=True)
df_usr_rate.to_csv(movie_usr_rate_path, index=False)

In [ ]:
df_usr_rate = pd.read_csv(movie_usr_rate_path)